# Introduction

Данный скрипт делает предобработку датасета, производит лемматизацию/стемминг текста, создаёт разметку на основе тегов

news_main_2020 -> news_main_prepr_2020

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

md = '/content/drive/My Drive/Colab Notebooks/Novosti/'

Mounted at /content/drive


In [ ]:
import sklearn.cluster as sc
import gensim
import random
import pandas as pd
import numpy as np
import time
import datetime
from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv(md+'data/news.csv')

In [ ]:
data

,Unnamed: 0,tags,dt,main_text
0,0,['Экономика'],2020-01-01 00:07:00,В России повысили зарплаты. В России повышен м...
1,1,['Бывший СССР'],2020-01-01 00:27:00,Партия Порошенко потребовала запретить поставк...
2,2,['Экономика'],2020-01-01 00:35:00,В России подорожали алкогольные напитки и сига...
3,3,['Наука и техника'],2020-01-01 00:48:00,Рогозин назвал поклонников Маска «Свидетелями ...
4,4,['Россия'],2020-01-01 01:02:00,В России изменились правила регистрации новых ...
...,...,...,...,...
86788,86788,['Мир'],2020-12-31 23:08:00,Трамп раскрыл срок поставок американской вакци...
86789,86789,['Мир'],2020-12-31 23:29:00,США укрепили военную базу в Ираке в преддверии...
86790,86790,['Мир'],2020-12-31 23:35:00,ВОЗ рекомендовала вакцину Pfizer для экстренно...
86791,86791,['Наука и техника'],2020-12-31 23:38:00,Рогозин назвал чушью версию о причастности роб...


In [ ]:
months = {
    'января':'01',
    'февраля':'02',
    'марта':'03',
    'апреля':'04',
    'мая':'05',
    'июня':'06',
    'июля':'07',
    'августа':'08',
    'сентября':'09',
    'октября':'10',
    'ноября':'11',
    'декабря':'12'
}
months_numbers = list(months.values())

In [ ]:
def show_elbow_plot(model,vocab_m):
  scores = []
  for n_cl in tqdm(range(1,10)):
    km = sc.KMeans(init='k-means++',n_clusters=n_cl, random_state=0).fit(model.wv[vocab_m])
    scores+= [-km.score(model.wv[vocab_m])]

  plt.scatter(range(1,10),scores)
  plt.show()

In [ ]:
def take_from_model(model):
  vocab_m = np.array(list(model.wv.vocab))
  counts_m = np.array([model.wv.vocab[v].count for v in vocab_m])
  n=10 # Выводит первые n самых частотных слов
  counts_max_m = sorted(np.unique(sorted(counts_m,reverse=True)[:n]),reverse=True)
  freq_index = [j  for i in counts_max_m for j in np.where(counts_m == i)[0]]
  freq_m = [vocab_m[i] for i in freq_index]
  return vocab_m, counts_m, counts_max_m, freq_m

# Предобработка датасета

# _обработка текста

In [ ]:
import string
import re
import threading
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import RussianStemmer
import nltk
rs = RussianStemmer()

In [ ]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Unzipping tokenizers/punkt.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 53.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=25dd98968706608ff4fb6ec4bb1eadd2040ceb0c1c51ff2f26037e426aa9528c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
stop_words = stopwords.words('russian')
stop_words.extend(['что','однако',
                   'это', 'так', 'вот', 'быть','уже', 'как', 'в','из-за', '—', '–', 'к', 'на', '...','»', '«','``',"''"])
stop_words.extend(list(string.punctuation))


def digit_flag(word):
  if re.search(r'\d+', word) is not None:
    if re.search(r'№\d+\b', word) is not None:
      word = 'число_номер'
    elif re.search(r'\b\d+-лет', word) is not None:
      word = 'число_возраст'
    elif re.search(r'\bтоп-\d+\b', word) is not None:
      word = 'число_топ'
    elif re.search(r'\b[0-2]\d:[0-5]\d\b', word) is not None:
      word = 'число_время'
    elif re.search(r'\b((19|20)\d\d)\b', word) is not None:
      word = 'число_год_' + re.search(r'\b((19|20)\d\d)\b', word)[0]
    else:
      word = 'число'
  return word


def lemmatize(word):
  p = morph.parse(word)[0]
  res = p.normal_form
  return res


def tokenize_sent2words_ru(sentence, stem_it = True):
    sentence=sentence.lower()
    tokens = word_tokenize(sentence, 'russian')
    if stem_it:
      tokens = [rs.stem(token) for token in tokens if (token not in stop_words)]
    else:
      tokens = [lemmatize(token) for token in tokens if (token not in stop_words)]
    tokens = [digit_flag(token) for token in tokens]
    return tokens


def tokenize_text2words_ru(text, stem_it = True):
    return [tokenize_sent2words_ru(sent, stem_it) for sent in sent_tokenize(text, 'russian')]

In [ ]:
data_main_text = data['main_text']
data_text_preprocessing = [tokenize_text2words_ru(main_text, False) for main_text in tqdm(data_main_text)]

  0%|          | 0/86793 [00:00<?, ?it/s]

In [ ]:
data = pd.read_pickle(md+'data/news_main_prepr_2020')

In [ ]:
data

,Unnamed: 0,tags,dt,lemm_text
0,0,['Экономика'],2020-01-01 00:07:00,"[[россия, повысить, зарплата], [россия, повыси..."
1,1,['Бывший СССР'],2020-01-01 00:27:00,"[[партия, порошенко, потребовать, запретить, п..."
2,2,['Экономика'],2020-01-01 00:35:00,"[[россия, подорожать, алкогольный, напиток, си..."
3,3,['Наука и техника'],2020-01-01 00:48:00,"[[рогозин, назвать, поклонник, маска, свидетел..."
4,4,['Россия'],2020-01-01 01:02:00,"[[россия, измениться, правило, регистрация, но..."
...,...,...,...,...
86788,86788,['Мир'],2020-12-31 23:08:00,"[[трамп, раскрыть, срок, поставка, американски..."
86789,86789,['Мир'],2020-12-31 23:29:00,"[[сша, укрепить, военный, база, ирак, преддвер..."
86790,86790,['Мир'],2020-12-31 23:35:00,"[[воз, рекомендовать, вакцина, pfizer, экстрен..."
86791,86791,['Наука и техника'],2020-12-31 23:38:00,"[[рогозин, назвать, чушь, версия, причастность..."


In [ ]:
data_text_preprocessing[0]

[['россия', 'повысить', 'зарплата'],
 ['россия', 'повысить', 'минимальный', 'размер', 'оплата', 'труд', 'мрот'],
 ['соответствовать',
  'закон',
  'конец',
  'декабрь',
  'подписать',
  'президент',
  'владимир',
  'путин'],
 ['повышение', 'составить', 'число', 'рубль', 'число', 'процент'],
 ['число',
  'январь',
  'федеральный',
  'мрот',
  'трудоспособный',
  'население',
  'составлять',
  'число',
  'рубль',
  'месяц'],
 ['закон',
  'размер',
  'мрот',
  'устанавливаться',
  'ежегодно',
  'ниже',
  'величина',
  'прожиточный',
  'минимум',
  'второй',
  'квартал',
  'предыдущий',
  'год'],
 ['порядок', 'действовать', 'число_год_2018', 'год'],
 ['путин',
  'декабрь',
  'пообещать',
  'подумать',
  'изменение',
  'правило',
  'польза',
  'увеличение',
  'мрот',
  'размер',
  'подоходный',
  'налог',
  'который',
  'учитываться'],
 ['человек',
  'получать',
  'зарплата',
  'выше',
  'уровень',
  'мрот',
  'который',
  'вычет',
  'налог',
  'опускаться',
  'ниже']]

In [ ]:
data['lemm_text'] = np.array(data_text_preprocessing, dtype=object)

In [ ]:
data

,Unnamed: 0,tags,dt,main_text,lemm_text
0,0,['Экономика'],2020-01-01 00:07:00,В России повысили зарплаты. В России повышен м...,"[[россия, повысить, зарплата], [россия, повыси..."
1,1,['Бывший СССР'],2020-01-01 00:27:00,Партия Порошенко потребовала запретить поставк...,"[[партия, порошенко, потребовать, запретить, п..."
2,2,['Экономика'],2020-01-01 00:35:00,В России подорожали алкогольные напитки и сига...,"[[россия, подорожать, алкогольный, напиток, си..."
3,3,['Наука и техника'],2020-01-01 00:48:00,Рогозин назвал поклонников Маска «Свидетелями ...,"[[рогозин, назвать, поклонник, маска, свидетел..."
4,4,['Россия'],2020-01-01 01:02:00,В России изменились правила регистрации новых ...,"[[россия, измениться, правило, регистрация, но..."
...,...,...,...,...,...
86788,86788,['Мир'],2020-12-31 23:08:00,Трамп раскрыл срок поставок американской вакци...,"[[трамп, раскрыть, срок, поставка, американски..."
86789,86789,['Мир'],2020-12-31 23:29:00,США укрепили военную базу в Ираке в преддверии...,"[[сша, укрепить, военный, база, ирак, преддвер..."
86790,86790,['Мир'],2020-12-31 23:35:00,ВОЗ рекомендовала вакцину Pfizer для экстренно...,"[[воз, рекомендовать, вакцина, pfizer, экстрен..."
86791,86791,['Наука и техника'],2020-12-31 23:38:00,Рогозин назвал чушью версию о причастности роб...,"[[рогозин, назвать, чушь, версия, причастность..."


In [ ]:
data = data.drop(['main_text'],axis=1)

In [ ]:
data_with_tags = pd.read_pickle(md+'data/newsp.pkl')

In [ ]:
data['tags'] = data_with_tags.tags

In [ ]:
data

,Unnamed: 0,tags,dt,lemm_text
0,0,['Экономика'],2020-01-01 00:07:00,"[[россия, повысить, зарплата], [россия, повыси..."
1,1,['Бывший СССР'],2020-01-01 00:27:00,"[[партия, порошенко, потребовать, запретить, п..."
2,2,['Экономика'],2020-01-01 00:35:00,"[[россия, подорожать, алкогольный, напиток, си..."
3,3,['Наука и техника'],2020-01-01 00:48:00,"[[рогозин, назвать, поклонник, маска, свидетел..."
4,4,['Россия'],2020-01-01 01:02:00,"[[россия, измениться, правило, регистрация, но..."
...,...,...,...,...
86788,86788,['Мир'],2020-12-31 23:08:00,"[[трамп, раскрыть, срок, поставка, американски..."
86789,86789,['Мир'],2020-12-31 23:29:00,"[[сша, укрепить, военный, база, ирак, преддвер..."
86790,86790,['Мир'],2020-12-31 23:35:00,"[[воз, рекомендовать, вакцина, pfizer, экстрен..."
86791,86791,['Наука и техника'],2020-12-31 23:38:00,"[[рогозин, назвать, чушь, версия, причастность..."


In [ ]:
data.to_pickle(md+ 'data/news_main_prepr_2020')

# _обработка тегов

In [ ]:
main_tags_list = [
    "society",
    "economy",
    "sports",
    "technology",
    "science",
    "entertainment",
    "other"
]

main_tags_en2ru={
    "society" :       "общество, происшествия и политика",
    "economy" :       "экономика",
    "sports":         "спорт",
    "science":        "наука",
    "technology":     "технологии",
    "entertainment":  "новости культуры",
    "other":          "другое"
}

main_tags_indexer={
    "society":[],
    "economy":[],
    "sports":[],
    "technology":[],
    "science":[],
    "entertainment":[],
    "other":[]
}

main_tags_dict = dict(zip(main_tags_list,np.zeros(len(main_tags_list))))

In [ ]:
tags = []
for _ in  data.tags:
    tags+=[_]

tags = np.array(tags)
all_tags = np.array([j for t in tags for j in t])
uniq_tags = np.unique(all_tags) #1183
len(uniq_tags)

42

In [ ]:
tags

array(["['Экономика']", "['Бывший СССР']", "['Экономика']", ...,
       "['Мир']", "['Наука и техника']", "['Мир']"], dtype='<U21')

In [ ]:
data.tags

,tags
0,['Экономика']
1,['Бывший СССР']
2,['Экономика']
3,['Наука и техника']
4,['Россия']
...,...
86788,['Мир']
86789,['Мир']
86790,['Мир']
86791,['Наука и техника']


In [ ]:
uniq_tags_counter_dict = dict(zip(uniq_tags, np.zeros(len(uniq_tags)).astype('int64')))
for i in all_tags:
    uniq_tags_counter_dict[i]+=1

uniq_tags_counter_list = np.array(list(uniq_tags_counter_dict.values()))

In [ ]:
uniq_tags[np.where(uniq_tags_counter_list >3000)]

array([' ', "'", '[', ']', 'Б', 'Д', 'И', 'К', 'М', 'Н', 'П', 'Р', 'С',
       'Ц', 'Э', 'а', 'в', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н',
       'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ш', 'ы', 'ь', 'я'], dtype='<U1')

# __теги лента


In [ ]:
def take_first_v_tags(data):
  # will return:
    # tags,
    # all_tags,
    # uniq_tags,
    # uniq_tags_counter_dict,
    # uniq_tags_counter_list

  tags = []
  for _ in  data.tags:
      tags+=[_]
  tags = np.array(tags)
  all_tags = np.array([j for t in tags for j in t])
  uniq_tags = np.unique(all_tags) #1183
  print(f'len uniq_tags - {len(uniq_tags)}')

  uniq_tags_counter_dict = dict(zip(uniq_tags, np.zeros(len(uniq_tags)).astype('int64')))
  for i in all_tags:
      uniq_tags_counter_dict[i]+=1
  uniq_tags_counter_list = np.array(list(uniq_tags_counter_dict.values()))

  return tags, all_tags, uniq_tags, uniq_tags_counter_dict, uniq_tags_counter_list

In [ ]:
def print_most_simular_tags(tag_item, top_n_tags = 20):
  # выводит в print топ под-тегов данного тега

  print(f' __now_tags - {tag_item[0]} - {tag_item[1]}')
  # now_all_tags = np.array([])
  now_all_tags = []
  for tags_i in tags:
    if tag_item[0] in tags_i:
      # now_all_tags = np.append(now_all_tags, tags_i)
      now_all_tags.extend(tags_i)
  now_uniq_tags = np.unique(now_all_tags)
  print(f' _len uniq_tags - {len(now_uniq_tags)}:')

  now_uniq_tags_counter_dict = dict(zip(now_uniq_tags, np.zeros(len(now_uniq_tags)).astype('int64')))
  for i in now_all_tags:
    now_uniq_tags_counter_dict[i]+=1

  now_uniq_tags_counter_items = list(now_uniq_tags_counter_dict.items())
  now_uniq_tags_counter_items.sort(key=lambda i: i[1], reverse=True)
  for i in now_uniq_tags_counter_items[:top_n_tags]:
    percentage = i[1]/uniq_tags_counter_dict[i[0]]
    percentage = float("{0:.2f}".format(percentage))
    print(f'{i} {percentage}')
  print()

In [ ]:
df_ria = data[data.website == 'ria']
df_rbc = data[data.website == 'rbc']
df_len = data[data.website == 'lenta']

In [ ]:
# получение тегов Лента
tags, all_tags, uniq_tags, uniq_tags_counter_dict, uniq_tags_counter_list = take_first_v_tags(data)

len uniq_tags - 42


In [ ]:
uniq_tags

array([' ', "'", '-', '6', '9', '[', ']', 'Б', 'Д', 'И', 'К', 'М', 'Н',
       'О', 'П', 'Р', 'С', 'Ц', 'Э', 'а', 'в', 'е', 'ж', 'з', 'и', 'й',
       'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ц', 'ш',
       'ы', 'ь', 'я'], dtype='<U1')

In [ ]:
uniq_tags_counter_dict

{'69-я параллель': 483,
 'Бизнес': 2,
 'Бывший СССР': 7730,
 'Дом': 3300,
 'Из жизни': 4311,
 'Интернет и СМИ': 4459,
 'Культура': 4059,
 'Мир': 11068,
 'Наука и техника': 4852,
 'Нацпроекты': 2828,
 'Оружие': 2,
 'Путешествия': 3699,
 'Россия': 17846,
 'Силовые структуры': 4832,
 'Спорт': 5844,
 'Ценности': 3220,
 'Экономика': 8258}

In [ ]:
now_tag = 'Дом'
np.array(data[data.tags.apply(lambda x: now_tag in x)].main_text)[:10]

array(['Пьяный россиянин сбросил с балкона на автомобиль компьютер и ноутбуки. В Краснодаре пьяный россиянин сбросил на автомобиль с балкона 11-го этажа три ноутбука, системный блок и монитор. Об этом сообщают «Кубанские новости» со ссылкой на представителя МВД по региону Артема Коноваленко. По предварительным данным, вся техника принадлежала мужчине, поскольку он является компьютерным мастером. Она упала на припаркованный автомобиль «Хендай Солярис», водитель которого ушел гулять в местный парк. 46-летнего нарушителя задержали в квартире. Причину своих действий мужчина объяснять отказался. В настоящее время по данному факту проводится проверка и устанавливается сумма причиненного ущерба. На мужчину составили административный протокол за мелкое хулиганство, который предусматривает до 15 суток административного ареста. Ранее жителя Екатеринбурга признали виновным в жестоком обращении с животным за то, что он выбросил собаку с балкона пятого этажа. Такса получила множественные переломы с

In [ ]:
tags_mapping_len={
    "Экономика": "economy",
    "Спорт": "sports",
    "Силовые структуры": "society",
    "Бизнес": "economy",
    "Ценности": "other",
    "Из жизни": "other",
    "Дом": "other",
    "Культура": "entertainment",
    "Интернет и СМИ": "technology",
    "Наука и техника": "science",
}

In [ ]:
for k,v in tags_mapping_len.items():
  main_tags_dict[v]+=uniq_tags_counter_dict[k]
print(np.sum(list(list(main_tags_dict.values()))))
main_tags_dict

43137.0


{'economy': 8260.0,
 'entertainment': 4059.0,
 'other': 10831.0,
 'science': 4852.0,
 'society': 4832.0,
 'sports': 5844.0,
 'technology': 4459.0}

In [ ]:
tags_mapping_len

{'Бизнес': 'economy',
 'Дом': 'other',
 'Из жизни': 'other',
 'Интернет и СМИ': 'technology',
 'Культура': 'entertainment',
 'Наука и техника': 'science',
 'Силовые структуры': 'society',
 'Спорт': 'sports',
 'Ценности': 'other',
 'Экономика': 'economy'}

In [ ]:
for i, tags_i in tqdm(enumerate(tags)):
  for k,v in list(tags_mapping_len.items()):
    if type(k) == tuple:
      flag = True
      for k_i in list(k):
        if k_i not in tags_i:
          flag = False
          break

      if flag:
        main_tags_indexer[v].extend([now_df.iloc[i].name])
        break

    else:
      if k in tags_i:
        main_tags_indexer[v].extend([now_df.iloc[i].name])
        break


for k, v in main_tags_indexer.items():
  print(f'{k} {len(v)}')


society 4832
economy 8260
sports 5844
technology 4459
science 4852
entertainment 4059
other 10831


In [ ]:
main_tags_dict

{'economy': 8260.0,
 'entertainment': 4059.0,
 'other': 3220.0,
 'science': 0.0,
 'society': 4832.0,
 'sports': 5844.0,
 'technology': 4459.0}

# __теги риа


In [ ]:
now_df = df_ria
# получение тегов РИА
tags, all_tags, uniq_tags, uniq_tags_counter_dict, uniq_tags_counter_list = take_first_v_tags(now_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


len uniq_tags - 12273


In [ ]:
uniq_tags_counter_items = list(uniq_tags_counter_dict.items())
uniq_tags_counter_items.sort(key=lambda i: i[1], reverse=True)
uniq_tags_counter_items[:10]

[('В мире', 14129),
 ('Россия', 11315),
 ('Общество', 10663),
 ('Коронавирус COVID-19', 10525),
 ('Распространение коронавируса', 10403),
 ('Коронавирус в России', 6983),
 ('США', 5492),
 ('Происшествия', 5488),
 ('Москва', 5212),
 ('Здоровье - Общество', 3096)]

In [ ]:
now_tag = 'Бизнес'
now_tag_item = (now_tag, uniq_tags_counter_dict[now_tag])
print_most_simular_tags(now_tag_item,20)

 __now_tags - Бизнес - 11
 _len uniq_tags - 44:
('Бизнес', 11) 1.0
('Коронавирус в России', 6) 0.0
('Экономика', 6) 0.0
('Коронавирус COVID-19', 5) 0.0
('Общество', 5) 0.0
('Распространение коронавируса', 4) 0.0
('Владимир Путин', 3) 0.0
('Налоги', 2) 0.1
('BMW AG', 1) 0.11
('IT-компании', 1) 0.5
('Volkswagen Group', 1) 0.1
('Автомобили', 1) 0.05
('Анастасия Татулова', 1) 1.0
('Банки', 1) 0.02
('Безработица', 1) 0.12
('Высшая школа экономики (ВШЭ)', 1) 0.03
('Госдума РФ', 1) 0.0
('Детские вопросы', 1) 0.06
('Инфографика', 1) 0.01
('Ипотека', 1) 0.01



In [ ]:
for utci in tqdm(uniq_tags_counter_items[:1]):
  print_most_simular_tags(utci, 10)


 __now_tags - В мире - 14129
 _len uniq_tags - 4455:
('В мире', 14129) 1.0
('США', 3766) 0.69
('Россия', 3266) 0.29
('Распространение коронавируса', 3048) 0.29
('Коронавирус COVID-19', 2829) 0.27
('Украина', 1838) 0.79
('Белоруссия', 1529) 0.82
('Армения', 1193) 0.94
('Ситуация в Нагорном Карабахе', 1182) 0.96
('Дональд Трамп', 1101) 0.81




In [ ]:
tags_mapping_ria={
    'Протесты в Белоруссии': "society",
    ('Общество', 'Сергей Собянин'): "society",
    ('Общество', 'СН_Образование'): "society",
    ('Госдума РФ', 'Единая Россия'): "society",
    'Политика': "society",
    'Безопасность' : "society",
    'Происшествия' : "society",

    'Экономика': "economy",

    'Шоу-бизнес': "entertainment",
    'Новости культуры': "entertainment",
    'Культура': "entertainment",

    'Наука':"science",
    'Технологии':"science",
    'Космос - РИА Наука':"science",

    'Футбол': "sports",
    'Вокруг спорта': "sports",
    'Спорт': "sports",
    'Фигурное катание': "sports",
    'Олимпийские игры': "sports",
    'Хоккей': "sports",
    'Sport Stories': "sports",
    'UFC': "sports",
    'Единоборства': "sports",
    'Смешанные боевые искусства (ММА)': "sports",
    'Всемирное антидопинговое агентство (WADA)': "sports",
}

In [ ]:
for k,v in list(tags_mapping_ria.items()):
  if type(k) == tuple:
    # потом дописать и привести в норм вид

    main_tags_dict[v]+=uniq_tags_counter_dict[k[0]]
  else:
    main_tags_dict[v]+=uniq_tags_counter_dict[k]
print(np.sum(list(list(main_tags_dict.values()))))
main_tags_dict

87008.0


{'economy': 11077.0,
 'entertainment': 7725.0,
 'other': 10831.0,
 'science': 7280.0,
 'society': 35793.0,
 'sports': 9843.0,
 'technology': 4459.0}

In [ ]:
for i, tags_i in tqdm(enumerate(tags)):
  for k,v in list(tags_mapping_ria.items()):
    if type(k) == tuple:
      flag = True
      for k_i in list(k):
        if k_i not in tags_i:
          flag = False
          break

      if flag:
        main_tags_indexer[v].extend([now_df.iloc[i].name])
        break

    else:
      if k in tags_i:
        main_tags_indexer[v].extend([now_df.iloc[i].name])
        break


for k, v in main_tags_indexer.items():
  print(f'{k} {len(v)}')


society 13978
economy 11044
sports 8673
technology 4459
science 6782
entertainment 6389
other 10831


In [ ]:
main_tags_dict

{'economy': 2817.0,
 'entertainment': 3666.0,
 'other': 0.0,
 'science': 2428.0,
 'society': 30961.0,
 'sports': 3999.0,
 'technology': 0.0}

# __теги рбк


In [ ]:
now_df = df_rbc
# получение тегов РИА
tags, all_tags, uniq_tags, uniq_tags_counter_dict, uniq_tags_counter_list = take_first_v_tags(now_df)

len uniq_tags - 7009


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [ ]:
uniq_tags_counter_items = list(uniq_tags_counter_dict.items())
uniq_tags_counter_items.sort(key=lambda i: i[1], reverse=True)
uniq_tags_counter_items[:100]

[('Общество', 13358),
 ('коронавирус', 5524),
 ('Политика', 4422),
 ('новости', 1765),
 ('Москва', 1729),
 ('новости РБК', 1526),
 ('Владимир Путин', 1122),
 ('США', 934),
 ('Белоруссия', 908),
 ('Россия', 646),
 ('Украина', 488),
 ('Дмитрий Песков', 481),
 ('Бизнес', 432),
 ('COVID-19', 420),
 ('Дональд Трамп', 404),
 ('Технологии и медиа', 402),
 ('ДТП', 400),
 ('вакцина', 399),
 ('пожар', 398),
 ('Санкт-Петербург', 388),
 ('погода', 380),
 ('пандемия', 369),
 ('Экономика', 348),
 ('протесты', 314),
 ('Александр Лукашенко', 310),
 ('суд', 296),
 ('Нагорный Карабах', 275),
 ('Роспотребнадзор', 267),
 ('Михаил Ефремов', 261),
 ('Армения', 260),
 ('Михаил Мишустин', 244),
 ('Китай', 243),
 ('Турция', 224),
 ('выборы', 223),
 ('Сергей Собянин', 220),
 ('Азербайджан', 219),
 ('Алексей Навальный', 218),
 ('карантин', 217),
 ('Крым', 212),
 ('Госдума', 207),
 ('убийство', 206),
 ('Минобороны', 195),
 ('СК', 194),
 ('смерть', 191),
 ('Финансы', 188),
 ('Минздрав', 181),
 ('задержание', 176),

In [ ]:
now_tag = 'Технологии'
now_tag_item = (now_tag, uniq_tags_counter_dict[now_tag])
print_most_simular_tags(now_tag_item,20)

KeyError: ignored

In [ ]:
for utci in tqdm(uniq_tags_counter_items[:1]):
  print_most_simular_tags(utci, 10)


 __now_tags - Общество - 13358
 _len uniq_tags - 5421:
('Общество', 13358) 1.0
('коронавирус', 4886) 0.88
('Москва', 1673) 0.97
('новости', 1544) 0.87
('новости РБК', 1388) 0.91
('Владимир Путин', 450) 0.4
('пожар', 398) 1.0
('ДТП', 397) 0.99
('COVID-19', 393) 0.94
('США', 386) 0.41




In [ ]:
tags_mapping_rbc={
    'Политика': "society",
    'ДТП': "society",
    'пожар': "society",
    'протесты': "society",
    'суд': "society",
    'выборы': "society",
    'Госдума': "society",
    'Минобороны': "society",
    'МЧС': "society",
    'полиция': "society",

    'Бизнес': "economy",
    'Экономика': "economy",

    'Технологии и медиа': "technology",

    'погода': "other",
    'дети': "other",

}

In [ ]:
for k,v in list(tags_mapping_rbc.items()):
  if type(k) == tuple:
    # потом дописать и привести в норм вид

    main_tags_dict[v]+=uniq_tags_counter_dict[k[0]]
  else:
    main_tags_dict[v]+=uniq_tags_counter_dict[k]
print(np.sum(list(list(main_tags_dict.values()))))
main_tags_dict

95502.0


{'economy': 11857.0,
 'entertainment': 7725.0,
 'other': 11358.0,
 'science': 7280.0,
 'society': 42578.0,
 'sports': 9843.0,
 'technology': 4861.0}

In [ ]:
for i, tags_i in tqdm(enumerate(tags)):
  for k,v in list(tags_mapping_rbc.items()):
    if type(k) == tuple:
      flag = True
      for k_i in list(k):
        if k_i not in tags_i:
          flag = False
          break

      if flag:
        main_tags_indexer[v].extend([now_df.iloc[i].name])
        break

    else:
      if k in tags_i:
        main_tags_indexer[v].extend([now_df.iloc[i].name])
        break


for k, v in main_tags_indexer.items():
  print(f'{k} {len(v)}')


society 20141
economy 11803
sports 8673
technology 4845
science 6782
entertainment 6389
other 11322


In [ ]:
l = list(main_tags_indexer.values())
print(len(l))
np.sum([len(l_i) for l_i in l])

7


57492

# __индексирование тегов

In [ ]:
l = list(main_tags_indexer.values())
print(np.sum([len(l_i) for l_i in l]))

print(np.unique([l_i for l_i in l]))

for k, v in main_tags_indexer.items():
  print(f'{k} {len(v)}')

69955
[list([0, 3, 13, 16, 23, 25, 34, 39, 46, 48, 67, 72, 123, 191, 221, 241, 251, 264, 273, 288, 313, 316, 359, 388, 392, 458, 477, 481, 518, 523, 529, 580, 605, 623, 653, 684, 694, 697, 704, 721, 777, 794, 827, 833, 862, 880, 882, 969, 1042, 1044, 1048, 1052, 1085, 1103, 1218, 1232, 1249, 1278, 1285, 1295, 1318, 1334, 1384, 1562, 1574, 1580, 1603, 1678, 1680, 1705, 1715, 1740, 1745, 1760, 1783, 1817, 1831, 1835, 1862, 1885, 1910, 1917, 1941, 1954, 1965, 1972, 1980, 1986, 1987, 2028, 2061, 2076, 2090, 2102, 2123, 2146, 2166, 2171, 2199, 2231, 2262, 2290, 2338, 2346, 2361, 2362, 2377, 2379, 2391, 2400, 2461, 2623, 2681, 2689, 2695, 2711, 2738, 2744, 2844, 2849, 2864, 2875, 2883, 2903, 2921, 2941, 2963, 2977, 3011, 3031, 3046, 3069, 3090, 3111, 3119, 3127, 3134, 3147, 3148, 3158, 3161, 3175, 3198, 3211, 3213, 3231, 3236, 3238, 3255, 3264, 3279, 3303, 3338, 3346, 3352, 3369, 3378, 3394, 3417, 3433, 3446, 3454, 3472, 3475, 3483, 3506, 3514, 3517, 3522, 3530, 3533, 3539, 3544, 3548, 3552,

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
all_indexes = []
for ll in [l_i for l_i in l]:
  all_indexes.extend(ll)

all_indexes.sort()
print(len(all_indexes))
uniq_indexes = np.unique(all_indexes)
len(uniq_indexes)

69955


69955

In [ ]:

print(all_indexes)

[0, 3, 4, 6, 13, 14, 15, 16, 17, 22, 23, 25, 28, 30, 31, 33, 34, 39, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 56, 60, 62, 64, 66, 67, 68, 70, 71, 72, 74, 78, 79, 84, 85, 86, 88, 89, 96, 97, 98, 99, 101, 104, 106, 107, 110, 113, 117, 118, 119, 123, 124, 125, 127, 128, 129, 130, 131, 133, 134, 135, 136, 138, 139, 140, 141, 142, 144, 146, 149, 150, 157, 158, 159, 160, 162, 163, 166, 168, 169, 171, 175, 178, 189, 191, 195, 197, 198, 199, 201, 206, 208, 212, 220, 221, 230, 231, 233, 235, 241, 242, 245, 251, 252, 253, 259, 260, 264, 265, 266, 267, 268, 269, 271, 272, 273, 274, 277, 281, 283, 286, 288, 290, 292, 293, 295, 297, 299, 301, 302, 303, 304, 309, 310, 311, 312, 313, 315, 316, 317, 320, 321, 323, 325, 331, 332, 335, 340, 343, 344, 345, 356, 359, 365, 370, 378, 385, 386, 387, 388, 389, 392, 396, 398, 400, 401, 405, 406, 412, 417, 418, 420, 422, 428, 430, 431, 432, 433, 436, 438, 447, 450, 451, 453, 454, 458, 464, 468, 472, 474, 475, 477, 481, 484, 486, 488, 491, 492, 495, 497, 498,

In [ ]:

print([l_i for l_i in l])
print(l.sort())

[[84, 85, 127, 131, 134, 281, 293, 295, 340, 428, 438, 491, 583, 615, 894, 900, 1054, 1068, 1074, 1089, 1099, 1180, 1207, 1241, 1269, 1333, 1586, 1746, 1794, 1823, 1825, 1836, 1840, 1906, 1922, 1950, 1998, 2108, 2127, 2145, 2173, 2182, 2202, 2249, 2279, 2554, 2565, 2575, 2703, 2720, 2816, 2822, 2834, 2845, 2892, 2925, 2950, 2989, 3040, 3073, 3081, 3091, 3125, 3190, 3218, 3232, 3315, 3337, 3356, 3362, 3385, 3415, 3426, 3438, 3440, 3450, 3473, 3541, 3679, 3712, 3789, 3799, 3813, 3823, 3973, 3988, 4054, 4061, 4103, 4138, 4139, 4148, 4167, 4185, 4198, 4203, 4219, 4245, 4278, 4296, 4298, 4312, 4315, 4342, 4394, 4515, 4527, 4550, 4568, 4645, 4706, 4766, 5078, 5083, 5088, 5152, 5277, 5291, 5304, 5329, 5373, 5391, 5416, 5444, 5474, 5519, 5559, 5668, 5699, 5754, 5769, 5778, 5779, 5827, 5838, 5844, 5851, 5889, 5897, 5964, 6131, 6158, 6169, 6173, 6175, 6222, 6228, 6261, 6303, 6331, 6334, 6354, 6513, 6518, 6523, 6532, 6589, 6608, 6635, 6658, 6662, 6676, 6704, 6765, 6804, 6842, 6917, 6919, 6922, 69

In [ ]:
tags_column = np.array(['']*len(data), dtype='<U13')

cross_counter = []

for k, v in list(main_tags_indexer.items()):
  for v_i in v:
    if tags_column[v_i]!='':
      cross_counter.extend(v_i)
    tags_column[v_i] = k


len(cross_counter)

0

In [ ]:
tags_column

array(['economy', '', '', ..., 'science', '', ''], dtype='<U13')

In [ ]:
data['main_tag'] = tags_column

In [ ]:
data = data.drop(['tags'],axis=1)

In [ ]:
data.to_pickle(md+ 'data/news_main_prepr_2020')

# stem/lemm датасеты

In [ ]:
data = pd.read_pickle(md+'data/news_main_prepr_2020')

In [ ]:
label = np.array(data.label)

In [ ]:
stem_text = []
for data_stem_text_i in tqdm(data.stem_text):
  stem_text_i = ''
  for dst in data_stem_text_i:
    for dst_i in dst:
      stem_text_i+=dst_i + ' '
  stem_text.extend([stem_text_i])
len(stem_text)

151841

In [ ]:
df = pd.DataFrame(stem_text)
df = df.rename(columns={0: "text"})
df['label'] = data.label
df.to_pickle(md + 'data/data_stem')

In [ ]:
data = pd.read_pickle(md+'data/data_lemm')

In [ ]:
label = np.array(data.label)

In [ ]:
lemm_text = []
for data_lemm_text_i in tqdm(data.lemm_text):
  lemm_text_i = ''
  for dlt in data_lemm_text_i:
    for dlt_i in dlt:
      lemm_text_i+=dlt_i + ' '
  lemm_text.extend([lemm_text_i])
len(lemm_text)

151841

In [ ]:
df = pd.DataFrame(lemm_text)
df = df.rename(columns={0: "text"})
df['label'] = data.label
df.to_pickle(md + 'data/data_lemm')

In [ ]:
data

,Unnamed: 0,tags,dt,lemm_text
0,0,['Экономика'],2020-01-01 00:07:00,"[[россия, повысить, зарплата], [россия, повыси..."
1,1,['Бывший СССР'],2020-01-01 00:27:00,"[[партия, порошенко, потребовать, запретить, п..."
2,2,['Экономика'],2020-01-01 00:35:00,"[[россия, подорожать, алкогольный, напиток, си..."
3,3,['Наука и техника'],2020-01-01 00:48:00,"[[рогозин, назвать, поклонник, маска, свидетел..."
4,4,['Россия'],2020-01-01 01:02:00,"[[россия, измениться, правило, регистрация, но..."
...,...,...,...,...
86788,86788,['Мир'],2020-12-31 23:08:00,"[[трамп, раскрыть, срок, поставка, американски..."
86789,86789,['Мир'],2020-12-31 23:29:00,"[[сша, укрепить, военный, база, ирак, преддвер..."
86790,86790,['Мир'],2020-12-31 23:35:00,"[[воз, рекомендовать, вакцина, pfizer, экстрен..."
86791,86791,['Наука и техника'],2020-12-31 23:38:00,"[[рогозин, назвать, чушь, версия, причастность..."


In [ ]:


# Индексирование тегов
data['tags_index'] = pd.factorize(data['tags'])[0]





In [ ]:
data = data.drop(columns=['Unnamed: 0'])

In [ ]:
data

,tags,dt,lemm_text,tags_index
0,['Экономика'],2020-01-01 00:07:00,"[[россия, повысить, зарплата], [россия, повыси...",0
1,['Бывший СССР'],2020-01-01 00:27:00,"[[партия, порошенко, потребовать, запретить, п...",1
2,['Экономика'],2020-01-01 00:35:00,"[[россия, подорожать, алкогольный, напиток, си...",0
3,['Наука и техника'],2020-01-01 00:48:00,"[[рогозин, назвать, поклонник, маска, свидетел...",2
4,['Россия'],2020-01-01 01:02:00,"[[россия, измениться, правило, регистрация, но...",3
...,...,...,...,...
86788,['Мир'],2020-12-31 23:08:00,"[[трамп, раскрыть, срок, поставка, американски...",4
86789,['Мир'],2020-12-31 23:29:00,"[[сша, укрепить, военный, база, ирак, преддвер...",4
86790,['Мир'],2020-12-31 23:35:00,"[[воз, рекомендовать, вакцина, pfizer, экстрен...",4
86791,['Наука и техника'],2020-12-31 23:38:00,"[[рогозин, назвать, чушь, версия, причастность...",2


In [ ]:
data.to_pickle(md+ 'data/news_main_prepr_2020_last')

In [ ]:
data = pd.read_pickle(md+'data/news_main_prepr_2020_last')

In [ ]:
data.head(50)

,tags,dt,lemm_text,tags_index
0,['Экономика'],2020-01-01 00:07:00,"[[россия, повысить, зарплата], [россия, повыси...",0
1,['Бывший СССР'],2020-01-01 00:27:00,"[[партия, порошенко, потребовать, запретить, п...",1
2,['Экономика'],2020-01-01 00:35:00,"[[россия, подорожать, алкогольный, напиток, си...",0
3,['Наука и техника'],2020-01-01 00:48:00,"[[рогозин, назвать, поклонник, маска, свидетел...",2
4,['Россия'],2020-01-01 01:02:00,"[[россия, измениться, правило, регистрация, но...",3
5,['Мир'],2020-01-01 01:14:00,"[[ким, чен, ын, пообещать, показать, новый, ор...",4
6,['Мир'],2020-01-01 01:34:00,"[[трамп, пригрозить, иран, нападение, посольст...",4
7,['Экономика'],2020-01-01 01:36:00,"[[россия, повысить, пенсия], [число, январь, ч...",0
8,['Экономика'],2020-01-01 02:03:00,"[[банковский, перевод, номер, телефон, стать, ...",0
9,['Спорт'],2020-01-01 02:20:00,"[[стать, известный, следующий, соперник, росси...",5


In [ ]:
df['tags'] = df['tags'].str.replace(r"[[]']", '', regex=True).str.strip()

In [ ]:
unique_values = data['tags'].unique()

print(unique_values)

["['Экономика']" "['Бывший СССР']" "['Наука и техника']" "['Россия']"
 "['Мир']" "['Спорт']" "['Культура']" "['Интернет и СМИ']"
 "['Силовые структуры']" "['Ценности']" "['Из жизни']" "['Путешествия']"
 "['Дом']" "['69-я параллель']" "['Нацпроекты']" "['Оружие']" "['Бизнес']"]


In [ ]:
class_counts = data['tags'].value_counts()

print("Количество элементов каждого класса:")
print(class_counts)


Количество элементов каждого класса:
tags
['Россия']               17846
['Мир']                  11068
['Экономика']             8258
['Бывший СССР']           7730
['Спорт']                 5844
['Наука и техника']       4852
['Силовые структуры']     4832
['Интернет и СМИ']        4459
['Из жизни']              4311
['Культура']              4059
['Путешествия']           3699
['Дом']                   3300
['Ценности']              3220
['Нацпроекты']            2828
['69-я параллель']         483
['Оружие']                   2
['Бизнес']                   2
Name: count, dtype: int64


In [ ]:
tags_mapping_len={
    "['Экономика']": "economy",
    "Спорт": "sports",
    "Силовые структуры": "society",
    "Бизнес": "economy",
    "Ценности": "life",
    "Из жизни": "life",
    "Дом": "life",
    "Культура": "entertainment",
    "Интернет и СМИ": "technology",
    "Наука и техника": "science",
    "Россия": "russia",
    "Мир": "society",
    "Бывший СССР": "history",
    "Путешествия": "entertainment",
    "Нацпроекты": "entertainment",
    "69-я параллель": "science",
    "Оружие": "society"
    }

In [ ]:
data['mapped_tags'] = data['tags'].map(tags_mapping_len)

NameError: name 'tags_num' is not defined

In [ ]:
data.head(50)

,tags,dt,lemm_text,tags_index,mapped_tags
0,['Экономика'],2020-01-01 00:07:00,"[[россия, повысить, зарплата], [россия, повыси...",0,economy
1,['Бывший СССР'],2020-01-01 00:27:00,"[[партия, порошенко, потребовать, запретить, п...",1,NaN
2,['Экономика'],2020-01-01 00:35:00,"[[россия, подорожать, алкогольный, напиток, си...",0,economy
3,['Наука и техника'],2020-01-01 00:48:00,"[[рогозин, назвать, поклонник, маска, свидетел...",2,NaN
4,['Россия'],2020-01-01 01:02:00,"[[россия, измениться, правило, регистрация, но...",3,NaN
5,['Мир'],2020-01-01 01:14:00,"[[ким, чен, ын, пообещать, показать, новый, ор...",4,NaN
6,['Мир'],2020-01-01 01:34:00,"[[трамп, пригрозить, иран, нападение, посольст...",4,NaN
7,['Экономика'],2020-01-01 01:36:00,"[[россия, повысить, пенсия], [число, январь, ч...",0,economy
8,['Экономика'],2020-01-01 02:03:00,"[[банковский, перевод, номер, телефон, стать, ...",0,economy
9,['Спорт'],2020-01-01 02:20:00,"[[стать, известный, следующий, соперник, росси...",5,NaN
